This notebook was created on October 4th, 2018

The idea is to be able to upload Tif files (which are different time points with the z-stacks stacked in one same tiff) and convert them into HDF5 format for using them later with BigData Viewer from Fiji. 

In [1]:
import numpy as np
import h5py 
from glob import glob
from skimage.io import imread
import matplotlib.pyplot as plt
import os

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### First we get filenames for our images

In [2]:
path_input = 'E:/Test-Arianne-SPIM-DATA/SPIM/Channel1Stacked'
path_output = 'E:/Test-Arianne-SPIM-DATA/SPIM/Channel1Stacked/HDF5_Python'
filenames = sorted(glob('%s/*.tif'%path_input))


### Load and inspect a sample image

In [3]:
im = imread(filenames[0]) # Sample image
print('Shape: ',im.shape)
print('Type: ',im.dtype)

Shape:  (180, 2048, 2048)
Type:  uint16


In [4]:
# Original dimensions of the Raw data
z_dim = im.shape[0]
y_dim = im.shape[1]
x_dim = im.shape[2]
t_dim = len(filenames)
channels = 1

### Make an HDF5 file and an HDF5 dataset called '/x' within that file

In [5]:
name_HDF5 = 'SPIMChannel1_2' # How will the HDF5 be called
f = h5py.File('%s/%s.hdf5'%(path_output,name_HDF5),'a') # Make an HDF5 file

In [6]:
# Perform Downsamplings in X-Y-Z Dimension
def downSample(a, resolution):
    window_z, window_y, window_x = resolution
    z, x, y = a.shape
    xr = np.arange(0, x, window_x)
    yr = np.arange(0, y, window_y)
    zr = np.arange(0, z, window_z)

    return np.add.reduceat(np.add.reduceat(np.add.reduceat(im, zr, axis=0, dtype=np.int16), \
                                                             yr, axis=1, dtype=np.int16), xr, axis = 2, dtype=np.int16)

In [7]:
# Perform splitting into small 3D blocks: Chunks
def cellChunk(a, subdivision):
    zi, yi, xi = subdivision
    z_dim, y_dim, x_dim = a.shape
    # Number of 3D Arrays (chunks)
    chunks = int((z_dim/zi)*(y_dim/yi)*(x_dim/xi))
    
    cell = np.zeros((zi, yi, xi, chunks))
    count = 0
    for x in range(int(x_dim/xi)):
        for y in range(int(y_dim/yi)):
            for z in range(int(z_dim/zi)):
                cell[:,:,:,count] = a[z*zi:zi*(z+1), y*yi:yi*(y+1), x*xi:xi*(x+1)]
                count += 1
                
    return cell, chunks

In [ ]:
data_type = f.create_group('__DATA_TYPES__')
dt = h5py.special_dtype(enum=('i8',{'FALSE': 0, 'TRUE': 1}))
data_type.create_dataset('Enum_Boolean',dtype=dt)


dp = h5py.special_dtype(vlen=str)
data_type.create_dataset('String_VariableLength', dtype=dp)


In [8]:
resolution = np.array([[1,1,1], [1,2,2],[2,4,4],[4,8,8]])
subdivision = np.array([[4,32,32], [16,16,16],[16,16,16],[16,16,16]])
for i in range(channels):
    sSS = f.create_group('s%0.2d'%i)
    #sub = sSS.create_group('subdivisions')
    sub_sub = sSS.create_dataset('subdivisions',data=subdivision, dtype=np.int32)
    #resol = sSS.create_group('resolutions')
    sub_resol = sSS.create_dataset('resolutions',data=resolution, dtype=np.float64)

In [9]:
for t in range(3):
    # Load each tif stack
    im = imread(filenames[t]) # Sample image
    
    # Create the groups where each time point will be saved
    tTTTT = f.create_group('t%0.4d'%(t+1))
    
    for s in range(channels):
        # Create thr subgroup for each source (e.g. channel)
        sSS = tTTTT.create_group('s%0.2d'%s)
        # Create the arrays for subdivisions and resolutions
        
        # Perform the downsamplings
        for i in range(np.shape(resolution)[0]): 
            # Mipmap level
            L = sSS.create_group('%d'%i)
            aux_ds = downSample(im, resolution[i])
            
            # Chunk the downsampled data
            #cells, chunks = cellChunk(im, subdivision[i])

            L.create_dataset('cells', data=aux_ds, chunks=(subdivision[i][0],subdivision[i][1],subdivision[i][2]),\
                             compression='gzip', compression_opts=6, scaleoffset=0)
f.close()        

### Insert our images one at a time into the HDF5 dataset

In [ ]:
out = f.require_dataset('/x',shape=(len(filenames),im.shape[0],im.shape[1],im.shape[2]), dtype=im.dtype)

for i,fn in enumerate(filenames):
    im = imread(fn)
    out[i,:,:,:] = im

In [ ]:
np.shape(out)

In [ ]:
out.name

### Link the XML to the HDF5

In [ ]:
xmlfh = open('%s/SPIMChannel1_hdf5.xml'%path, 'rb')
h5f.attrs['xml'] = xmlfh.read()
h5f.attrs['xml']

In [ ]:
xmldata = """<xml>
<something>
    <else>Text</else>
</something>
</xml>
"""

# Write the xml file...
f = h5py.File('%s/%s.hdf5'%(path,name_HDF5)) # Make an HDF5 file
str_type = h5py.new_vlen(str)
ds = f.create_dataset('%s/SPIMChannel1_hdf5.xml'%path, shape=(1,), dtype=str_type)
ds[:] = xmldata
f.close()